In [18]:
from transformers import BertModel, BertTokenizer
import torch
from sklearn.metrics.pairwise import cosine_similarity

In [19]:
# load the bert-base uncased tokenizer. Quick check what does "uncased" mean?
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

print(f'Length of BERT base vocabulary: {len(tokenizer.vocab)}')

Length of BERT base vocabulary: 30522


In [20]:
text = "A simple sentence!"

tokens = tokenizer.encode(text) # get token ids per BERT-base's vocabulary including punctuation
print(tokens)

[101, 1037, 3722, 6251, 999, 102]


In [21]:
# decode will re-construct the sentence with the added [CLS] and [SEP] token
tokenizer.decode(tokens)

'[CLS] a simple sentence! [SEP]'

In [22]:
text = "My friend told me about this class and I love it so far! She was right."

tokens = tokenizer.encode(text)
print(tokens)

[101, 2026, 2767, 2409, 2033, 2055, 2023, 2465, 1998, 1045, 2293, 2009, 2061, 2521, 999, 2016, 2001, 2157, 1012, 102]


In [23]:
# A nicer printouts of token ids and token strings

print(f'Text: {text}. Num Tokens: {len(tokens)}')
for t in tokens:
  print(f'Token: {t}, subword: {tokenizer.decode([t])}')

Text: My friend told me about this class and I love it so far! She was right.. Num Tokens: 20
Token: 101, subword: [CLS]
Token: 2026, subword: my
Token: 2767, subword: friend
Token: 2409, subword: told
Token: 2033, subword: me
Token: 2055, subword: about
Token: 2023, subword: this
Token: 2465, subword: class
Token: 1998, subword: and
Token: 1045, subword: i
Token: 2293, subword: love
Token: 2009, subword: it
Token: 2061, subword: so
Token: 2521, subword: far
Token: 999, subword: !
Token: 2016, subword: she
Token: 2001, subword: was
Token: 2157, subword: right
Token: 1012, subword: .
Token: 102, subword: [SEP]


In [24]:
# Sinan is not in our vocab :'(

'sinan' in tokenizer.vocab

False

In [25]:
text_with_unknown_words = 'Sinan loves a beautiful day'
tokens_with_unknown_words = tokenizer.encode(text_with_unknown_words)

# We see our sub words in action!
for t in tokens_with_unknown_words:
  print(f'Token: {t}, subword: {tokenizer.decode([t])}')

Token: 101, subword: [CLS]
Token: 8254, subword: sin
Token: 2319, subword: ##an
Token: 7459, subword: loves
Token: 1037, subword: a
Token: 3376, subword: beautiful
Token: 2154, subword: day
Token: 102, subword: [SEP]


In [26]:
tokenizer.encode('sinan')

[101, 8254, 2319, 102]

In [27]:
tokenizer.encode('an')

[101, 2019, 102]

So here, it basically said that ##an != an, its just part of the previous words

In [28]:
text_with_unknown_words = "Sinan is our instructor for this awesomesauce class"
tokens_with_unknown_words = tokenizer.encode(text_with_unknown_words)

for t in tokens_with_unknown_words:
  print(f'Token: {t}, subword: {tokenizer.decode([t])}')

Token: 101, subword: [CLS]
Token: 8254, subword: sin
Token: 2319, subword: ##an
Token: 2003, subword: is
Token: 2256, subword: our
Token: 9450, subword: instructor
Token: 2005, subword: for
Token: 2023, subword: this
Token: 12476, subword: awesome
Token: 23823, subword: ##sau
Token: 3401, subword: ##ce
Token: 2465, subword: class
Token: 102, subword: [SEP]


In [29]:
text = "My friend told me about this class and I love it so far! She was right."

# encode_plus gives us token ids, attention mask, and segment ids (A vs B). Useful for training time
tokens = tokenizer.encode_plus(text)
print(tokens)

{'input_ids': [101, 2026, 2767, 2409, 2033, 2055, 2023, 2465, 1998, 1045, 2293, 2009, 2061, 2521, 999, 2016, 2001, 2157, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [30]:
tokenizer(text) # calling the tokenizer directly does the same thing as encode_plus

{'input_ids': [101, 2026, 2767, 2409, 2033, 2055, 2023, 2465, 1998, 1045, 2293, 2009, 2061, 2521, 999, 2016, 2001, 2157, 1012, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [31]:
# python in the 6th token (don't forget the [CLS] token!)
python_pet = tokenizer.encode('I love my pet python')

# python is the 6th token (don't forget the [CLS] token!)
python_language = tokenizer.encode('I love coding in python')

In [33]:
model = BertModel.from_pretrained('bert-base-uncased')

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [34]:
# contextful embedding of 'python' in 'I love my pet python'
python_pet_embedding = model(torch.tensor(python_pet).unsqueeze(0))[0][:,5,:].detach().numpy()

# contextful embedding of 'python' in 'I love coding in python'
python_language_embedding = model(torch.tensor(python_language).unsqueeze(0))[0][:,5,:].detach().numpy()

# contextful embedding of 'snake' in 'snake'
snake_alone_embedding = model(torch.tensor(tokenizer.encode('snake')).unsqueeze(0))[0][:,1,:].detach().numpy()

# contextful embedding of 'programming' in 'programming'
programming_alone_embedding = model(torch.tensor(tokenizer.encode('programming')).unsqueeze(0))[0][:,1,:].detach().numpy()

In [35]:
python_pet_embedding.shape

(1, 768)

In [36]:
python_language_embedding.shape

(1, 768)

In [37]:
# Similarity of the representation of the word Python in a sentence about coding to the word snake
cosine_similarity(python_language_embedding, snake_alone_embedding)

array([[0.58434796]], dtype=float32)

In [38]:
# Similarity of the representation of the word Python in a sentence about pets to the word snake. More similar!
cosine_similarity(python_pet_embedding, snake_alone_embedding)

array([[0.6928656]], dtype=float32)

In [39]:
# Similarity of the representation of the word Python in a sentence about pets to the word programming
cosine_similarity(python_pet_embedding, programming_alone_embedding)

array([[0.49864388]], dtype=float32)

In [40]:
# Similarity of the representation of the word Python in a sentence about coding to the word programming. More similar!
cosine_similarity(python_language_embedding, programming_alone_embedding)

array([[0.56147444]], dtype=float32)